In [1]:
#summary
#web : server-client : url
#동적페이지 : URL 변화 없이 페이지의 데이터 수정 : json(str) > response.json() > DataFrame
#정적페이지 : URL 변화를 주어 페이지의 데이터 수정 : html(str) > BeautifulSoup > css selector > DataFrame
#selenium : 웹브라이저를 python 코드로 컨트롤해서 데이터 수집
#requests(동적페이지, API) > requests(정적페이지) > selenium(앞의 두 방식보다 크롤링 속도가 떨어짐)

#웹크롤링 절차
# 1. 웹 서비스 분석(개발자 도구), API 문서 : URL 
# 2. request() > response(data) : data(json(str), html(str))
# 3. data(json(str), html(str)) > response.json(), BeautifulSoup(css-selector) > DataFrame

# request 할때 401, 403, 500 등등의 에러가 발생하는 경우 > headers 수정해서 데이터 요청(user-agent, refere)
# API 이용 : request token 수집 후 크롤링

### Zigbang 원룸 매물 데이터 수집
- 동이름 > 매물 데이터

In [2]:
import pandas as pd
import requests

#### Process
    - 동이름으로 위도 경도 구하기
    - 위도 경도로 geohash 알아내기
    - geohash로 매물 아이디 가져오기
    - 매물 아이디로 매물 정보 가져오기

#### 1. 동이름으로 위도 경도 구하기

In [10]:
# 1. URL
address='정자동'
url = f'https://apis.zigbang.com/v2/search?leaseYn=N&q={address}&serviceType=원룸' #leasn이라고 써있는 곳의 URL을 가져옴

# 2. request > response : json(str)
response = requests.get(url)
response
# 3. json(str) > lat, lng
data=response.json()['items'][0]
lat, lng = data['lat'], data['lng']
lat, lng

(37.361507415771484, 127.1114730834961)

In [9]:
response.json()['items'][0]

lat, lng = data['lat'], data['lng']
lat, lng

{'id': 650,
 'type': 'address',
 'name': '정자동',
 'hint': '',
 'description': '경기도 성남시 분당구 정자동',
 'lat': 37.361507415771484,
 'lng': 127.1114730834961,
 'zoom': 5,
 'polygon': [],
 '_score': None,
 '_source': {'name_length': 3,
  'local1': '경기도',
  'local2': '성남시 분당구',
  'local3': '정자동',
  'web_level': 15,
  'web_lat': 37.361507415771484,
  'web_lng': 127.1114730834961,
  'app_level': 15,
  'app_lat': 37.361507415771484,
  'app_lng': 127.1114730834961,
  '법정동코드': '4113510300'},
 'zoom_level': {'google': 15, 'daum': 4},
 'zoom_level_v2': {'app': 5, 'web': 4}}

In [5]:
response.text

'{"success":true,"code":"200","items":[{"id":650,"type":"address","name":"정자동","hint":"","description":"경기도 성남시 분당구 정자동","lat":37.361507415771484,"lng":127.1114730834961,"zoom":5,"polygon":[],"_score":null,"_source":{"name_length":3,"local1":"경기도","local2":"성남시 분당구","local3":"정자동","web_level":15,"web_lat":37.361507415771484,"web_lng":127.1114730834961,"app_level":15,"app_lat":37.361507415771484,"app_lng":127.1114730834961,"법정동코드":"4113510300"},"zoom_level":{"google":15,"daum":4},"zoom_level_v2":{"app":5,"web":4}},{"id":743,"type":"address","name":"정자동","hint":"","description":"경기도 수원시 장안구 정자동","lat":37.301753997802734,"lng":126.9908447265625,"zoom":5,"polygon":[],"_score":null,"_source":{"name_length":3,"local1":"경기도","local2":"수원시 장안구","local3":"정자동","web_level":15,"web_lat":37.301753997802734,"web_lng":126.9908447265625,"app_level":15,"app_lat":37.301753997802734,"app_lng":126.9908447265625,"법정동코드":"4111113000"},"zoom_level":{"google":15,"daum":4},"zoom_level_v2":{"app":5,"web":4}},{

#### 2. 위도 경도로 geohash 알아내기
geohash2 : pip install geohash2

In [12]:
pip install geohash2

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for geohash2: filename=geohash2-1.1-py3-none-any.whl size=15545 sha256=64252ea6c14f7148f4a6d61234b912c32d3ed1acb23f50592ff322b69eb0e108
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\ec\29\ab\aa2b0265ceaef06eac4aed2ca6ae1b17eb2ffb14d611f20172
Successfully built geohash2
Note: you may need to restart the kernel to use updated packages.


In [13]:
import geohash2

In [14]:
#precision : 클수록 영역이 작아짐
geohash = geohash2.encode(lat, lng, precision=5)
geohash

'wydks'

#### 3. geohash로 매물 아이디 가져오기

In [19]:
#items로 시작하는 곳의 URL을 디코딩해서 가져옴
url = f'https://apis.zigbang.com/v2/items?deposit_gteq=0&domain=zigbang&geohash={geohash}&needHasNoFiltered=true&rent_gteq=0&sales_type_in=전세|월세&service_type_eq=원룸'
url

response = requests.get(url)
response

<Response [200]>

In [34]:
data=response.json()['items']
ids = [item['item_id'] for item in data]
len(ids), ids[:5]

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,계약면적,room_type_title,floor,floor_string,building_floor,title,is_first_movein,room_type,address,random_location,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new,contract
317,None,35421581,https://ic.zigbang.com/ic/items/35421581/1.jpg,월세,월세,15000,40,64.99,"{'m2': 64.99, 'p': '19.7'}","{'m2': 26.76, 'p': '8.1'}",None,None,고,고,14,주거용. 전입가능 . 밝은동향. 풀옵션. 판교초역세권.,None,01,성남시 분당구 삼평동,"{'lat': 37.396638200150875, 'lng': 127.1116075...",False,True,오피스텔,[추천],경기도 성남시 분당구 삼평동,None,None,15,2023-02-14T12:33:30+09:00,False,
318,None,35442050,https://ic.zigbang.com/ic/items/35442050/1.jpg,월세,월세,1000,100,64.28,"{'m2': 64.28, 'p': '19.4'}","{'m2': 26.45, 'p': '8'}",None,None,저,저,14,"전입, 밝은집, 판교역도보3분",None,01,성남시 분당구 삼평동,"{'lat': 37.396657957337915, 'lng': 127.1122617...",False,True,오피스텔,[추천],경기도 성남시 분당구 삼평동,None,None,15,2023-02-15T14:23:04+09:00,True,


#### 4. 매물 아이디로 매물 정보 가져오기

In [24]:
#list라고 써있는 곳에서 URL 가져옴
url = 'https://apis.zigbang.com/v2/items/list'

params={
    'domain' : 'zigbang',
    'withCoalition' : 'true',
    'item_ids' : ids[:900],
}

response = requests.post(url, params)
response

<Response [200]>

In [30]:
pd.options.display.max_columns=50

In [36]:
data = response.json()['items'] 
df = pd.DataFrame(data) 
df.tail(2)

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,계약면적,room_type_title,floor,floor_string,building_floor,title,is_first_movein,room_type,address,random_location,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new,contract
317,None,35421581,https://ic.zigbang.com/ic/items/35421581/1.jpg,월세,월세,15000,40,64.99,"{'m2': 64.99, 'p': '19.7'}","{'m2': 26.76, 'p': '8.1'}",None,None,고,고,14,주거용. 전입가능 . 밝은동향. 풀옵션. 판교초역세권.,None,01,성남시 분당구 삼평동,"{'lat': 37.396638200150875, 'lng': 127.1116075...",False,True,오피스텔,[추천],경기도 성남시 분당구 삼평동,None,None,15,2023-02-14T12:33:30+09:00,False,
318,None,35442050,https://ic.zigbang.com/ic/items/35442050/1.jpg,월세,월세,1000,100,64.28,"{'m2': 64.28, 'p': '19.4'}","{'m2': 26.45, 'p': '8'}",None,None,저,저,14,"전입, 밝은집, 판교역도보3분",None,01,성남시 분당구 삼평동,"{'lat': 37.396657957337915, 'lng': 127.1122617...",False,True,오피스텔,[추천],경기도 성남시 분당구 삼평동,None,None,15,2023-02-15T14:23:04+09:00,True,


In [37]:
df.columns

Index(['section_type', 'item_id', 'images_thumbnail', 'sales_type',
       'sales_title', 'deposit', 'rent', 'size_m2', '공급면적', '전용면적', '계약면적',
       'room_type_title', 'floor', 'floor_string', 'building_floor', 'title',
       'is_first_movein', 'room_type', 'address', 'random_location', 'is_zzim',
       'status', 'service_type', 'tags', 'address1', 'address2', 'address3',
       'manage_cost', 'reg_date', 'is_new', 'contract'],
      dtype='object')

In [39]:
colums = ['item_id', 'sales_type', 'deposit', 'rent', 'size_m2', 'floor', 'building_floor', 'title', 'address', 'status', 'service_type', 'tags', 'address1', 'manage_cost'] 
df = df[colums] 
df = df[df['address'].str.contains(address)].reset_index(drop=True) 
df.tail(2)

,item_id,sales_type,deposit,rent,size_m2,floor,building_floor,title,address,status,service_type,tags,address1,manage_cost
209,35417677,월세,1500,80,59.50,2,3,올수리 정자역도보10분거리 넓은집,성남시 분당구 정자동,True,빌라,[],경기도 성남시 분당구 정자동,5
210,35284366,전세,23000,0,61.16,2,3,정자역.수내역 인근 넓고 컨디션좋은2룸.,성남시 분당구 정자동,True,빌라,[],경기도 성남시 분당구 정자동,5


In [40]:
#pep documents
#pep20, pep8 공식 코드작성 방법(일부 실무에서 pep따라서 안하면 업데이트가 안됨, 교정 프로그램 있음)
#flake8 : 코드효율성 체크(일부 비효율적인 코드로 나오면 업데이트가 안됨) 
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [ ]:
#문법(에러발생, 코드실행X), 컨벤션(에러발생X, 코드실행o)
#1data=1  문법이 틀림

In [ ]:
data=1 #컨벤션-연산자 앞뒤로 한칸씩 띄워야됨

In [41]:
#스페셜 커멘드 : %, %% : ipython에서 제공하는 커멘드
%pwd

'c:\\Users\\User\\webC\\20230216\\20230216'

In [42]:
%%time
data=1

CPU times: total: 0 ns
Wall time: 1.01 ms


In [ ]:
#쉘 커멘드 : ! : system에서 제공하는 커멘드
#!pip install geohash2